In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

In [2]:
unida = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/nacion_unida.json')
verif = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/verificado_2018.json')

In [3]:
print(verif.shape)

(169, 2)


In [4]:
unida['title_list'] = unida['title'].apply(lambda x: str(x).split(' '))
unida['filter_title'] = unida['title_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [5]:
verif['title_list'] = verif['title'].apply(lambda x: str(x).split(' '))
verif['filter_title'] = verif['title_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])

In [6]:
unida['filter_title_join'] = unida['filter_title'].apply(lambda x: ' '.join(x))
verif['filter_title_join'] = verif['filter_title'].apply(lambda x: ' '.join(x))

In [7]:
unida_str = unida['filter_title_join'].tolist()
verif_str = verif['filter_title_join'].tolist()

In [8]:
corpus = unida_str + verif_str

In [9]:
vectorizer = TfidfVectorizer()

In [10]:
matrix = vectorizer.fit_transform(corpus)

In [11]:
names = vectorizer.get_feature_names()

In [12]:
mat_uni = vectorizer.transform(unida_str)

In [13]:
mat_ver = vectorizer.transform(verif_str)

In [14]:
df_mat_uni = pd.DataFrame(mat_uni.todense(), columns=names)

In [15]:
df_mat_ver = pd.DataFrame(mat_ver.todense(), columns=names)

In [16]:
lst_mat_uni = df_mat_uni.values.tolist()
lst_mat_ver = df_mat_ver.values.tolist()

In [17]:
def simple_dot(a, b):
    dsum = 0
    for ((idx,), val) in np.ndenumerate(a):
        dsum += float(val) * float(b[idx])
    return(dsum)

In [18]:
def l2_norm(a):
    return(math.sqrt(np.dot(a, a)))

In [19]:
def cosine_similarity(a, b):
    return(np.dot(a,b) / (l2_norm(a) * l2_norm(b)))

In [20]:
maybe = []

for i in range(len(lst_mat_uni)):
    sim = cosine_similarity(np.array(lst_mat_uni[i]), np.array(lst_mat_ver[72]))
    maybe.append(sim)

In [21]:
m_maybe = max(maybe)
print(m_maybe)

0.3758322476009757


In [22]:
index = [i for i, j in enumerate(maybe) if j == m_maybe]

In [23]:
print(index)

[931]


In [24]:
print(verif['filter_title'][72])
print(unida['filter_title'][931])

['estosipaso', 'campana', 'anaya', 'si', 'contrato', 'encuestadora', 'massive', 'caller']
['anaya', 'paga', '4', 'millones', '640', 'mil', 'pesos', 'ecuestadora', 'massive', 'caller']


In [25]:
print(cosine_similarity(np.array(lst_mat_uni[931]), np.array(lst_mat_ver[72])))

0.3758322476009757


In [26]:
print(verif['title'][72])
print(unida['title'][931])

estosipaso la campana de anaya si contrato a la encuestadora massive caller
anaya paga 4 millones 640 mil pesos ecuestadora massive caller
